In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

In [4]:
ds = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)


In [17]:
ds['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [6]:
ds_tok = ds.map(tok, batched=True)
ds_tok

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [7]:
ds_tok['train']['input_ids'][0][:10]

[101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026]

In [8]:
ds_tok = ds_tok.remove_columns(["text"])
ds_tok = ds_tok.rename_column("label", "labels")

In [19]:
ds_tok

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [9]:
ds_tok.set_format("torch")

In [23]:
ds_tok['train'][0]

{'labels': tensor(0),
 'input_ids': tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
          2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
          2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
          2657,  2008,  2012,  2034,  2009,  2001,  8243,  2011,  1057,  1012,
          1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
          2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
          6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,  2005,  2870,
          1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1996,
          5436,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  3076,  2315,
         14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055,  2166,
          1012,  1999,  3327,  2016,  4122,  2000,  3579,  2014,  3086,  2015,
          2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
         25430, 1

In [10]:

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [11]:
acc = evaluate.load("accuracy")
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return acc.compute(predictions=preds, references=p.label_ids)

In [12]:
import torch
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


Device: cuda


In [13]:
args = TrainingArguments(
    output_dir="out_imdb",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=1,

    eval_strategy="steps",   # ✅ renamed
    eval_steps=500,
    save_steps=500,
    logging_steps=50,

    fp16=True  # only if NVIDIA GPU
)


Train and test on 5K data

In [14]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset=ds_tok["test"].select(range(2000)),
    compute_metrics=compute_metrics
)


In [15]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss,Validation Loss


TrainOutput(global_step=313, training_loss=0.3697529996926792, metrics={'train_runtime': 157.0666, 'train_samples_per_second': 31.834, 'train_steps_per_second': 1.993, 'total_flos': 331168496640000.0, 'train_loss': 0.3697529996926792, 'epoch': 1.0})

In [16]:
results = trainer.evaluate(eval_dataset=ds_tok["test"])
print(results)


{'eval_loss': 0.282593309879303, 'eval_accuracy': 0.88372, 'eval_runtime': 40.7957, 'eval_samples_per_second': 612.809, 'eval_steps_per_second': 19.169, 'epoch': 1.0}


Train and test it on entire data i:e; 25K

In [21]:
args = TrainingArguments(
    output_dir="out_imdb",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=1,

    eval_strategy="steps",   # ✅ renamed
    eval_steps=500,
    save_steps=500,
    logging_steps=50,

    fp16=True  # only if NVIDIA GPU
)


In [23]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset=ds_tok["test"].select(range(10000)),
    compute_metrics=compute_metrics
)


In [24]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=313, training_loss=0.13099152516252316, metrics={'train_runtime': 48.041, 'train_samples_per_second': 104.078, 'train_steps_per_second': 6.515, 'total_flos': 331168496640000.0, 'train_loss': 0.13099152516252316, 'epoch': 1.0})

In [25]:
results = trainer.evaluate(eval_dataset=ds_tok["test"])
print(results)


{'eval_loss': 0.36346563696861267, 'eval_accuracy': 0.9048, 'eval_runtime': 43.7041, 'eval_samples_per_second': 572.029, 'eval_steps_per_second': 17.893, 'epoch': 1.0}


In [26]:
!pip install nbstripout
!nbstripout FineTuneLLM.ipynb

Could not strip 'FineTuneLLM.ipynb': file not found
